# Machine Learning
## Trabajo MDS - 2o. Trim. ==> Inmigracion en Chile

**Profesor:** Graells, E..<br>
**Alumnos:** Briceno, H.; Rovai, M.; Tessada G.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline  

import warnings
warnings.filterwarnings("ignore")

## 1. Importar el dataset

Vamos a usar datos limpios obtenidos a partir del notebook: [Análisis exploratorio de los datos: 01-VISA-EDA.ipynb](https://github.com/Mjrovai/UDD_Master_Data_Science/blob/master/Trabajo%20-%20Inmigracion/01-VISA-EDA.ipynb). 

El Departamanto de Extranjeria y Migracion de Chile, aclara que si bien las bases se presentan de manera separada por tratarse de distintos tipos de permiso, ellas no constituyen universos distintos de inmigrantes. Un inmigrante en Chile para obtener una permanencia definitiva DEBE haber obtenido con anterioridad una visa, por lo que las bases de permanencias definitivas se conforman de personas que están contabilizadas en las visas. <br>
<br>Así, para una mejor lectura de los datos haremos una distinción en términos de migración transitoria y migración permanente. Se considerará que las visas corresponden a migración transitoria, pues los permisos duran entre uno y dos años. Las personas que sean titulares de una visa, para quedarse por un plazo mayor a dos años, deben solicitar un permiso de permanencia definitiva, de otra manera deben abandonar el país.<br>
- En este trabajo focaremos en la MIGRACIÓN DEFINITIVA, que están relacionadas a las Permisos de Residencia Definitiva
- Y se complementará esa información con la migración reciente, es decir, las visas otorgadas los dos últimos años (2017/2018).

**Información constante del dataset final** <br><br>
Las planillas de permisos migratorios contienen los siguientes datos, los cuales son extraídos de los formularios establecidos para solicitar residencia, disponibles en la página web del Departamento de Extranjería y Migración, de los cuales utilizamos:
- PAIS	     (País de origen)
- SEXO	
- EDAD	     (la edad en el año que recibió el benefício)
- ESTUDIOS	
- ACTIVIDAD	
- BENEFICIO	 (Visa temporária o definitiva)
- ANIO	     (año en que recibió el benefício)
- COMUNA

In [4]:
!ls ./data

inmigrantes_data_comunas_rm_4ML.xlsx inmigrantes_rm_comuna.xlsx


In [5]:
df = pd.read_excel('./data/inmigrantes_data_comunas_rm_4ML.xlsx')
df.shape #(493638, 8)

(493638, 8)

In [22]:
df.head(2)

,PAIS,SEXO,EDAD,ESTUDIOS,ACTIVIDAD,BENEFICIO,ANIO,COMUNA
0,peru,femenino,32,no informa,empleado,definitiva,2006,santiago
1,ecuador,masculino,58,no informa,empleado,definitiva,2007,providencia


In [23]:
df.COMUNA.value_counts()

santiago               139050
independencia           31260
las condes              30910
estacion central        25981
recoleta                24026
providencia             18646
nunoa                   16559
quilicura               14982
quinta normal           13083
la florida              11654
maipu                   10203
san bernardo             9376
san miguel               8619
penalolen                8477
conchali                 8270
puente alto              7569
colina                   7472
lo barnechea             7050
melipilla                6960
vitacura                 6552
macul                    6216
cerro navia              6018
pudahuel                 5832
lo prado                 5754
san joaquin              5267
la cisterna              5208
pedro aguirre cerda      4982
renca                    4280
cerrillos                4090
lo espejo                4034
la reina                 3931
huechuraba               3570
lampa                    3554
el bosque 

### Converter los labels de y en codigos de comunas

## 2. Preparar la data: 

### Dividir la data, sacando la variable target

In [30]:
X = df.copy()
y = X.pop('COMUNA').values

In [31]:
y

array(['santiago', 'providencia', 'las condes', ..., 'recoleta',
       'independencia', 'lampa'], dtype=object)

In [36]:
X.head(2)

,PAIS,SEXO,EDAD,ESTUDIOS,ACTIVIDAD,BENEFICIO,ANIO
0,peru,femenino,32,no informa,empleado,definitiva,2006
1,ecuador,masculino,58,no informa,empleado,definitiva,2007


In [37]:
X.PAIS.value_counts()

peru                          133949
venezuela                     108870
haiti                          77041
colombia                       60876
bolivia                        16164
argentina                      15983
ecuador                        15055
china republica popular         8738
espana                          8200
brasil                          7048
republica dominicana            5909
cuba                            5467
estados unidos                  3931
uruguay                         2905
mexico                          2829
francia                         2446
paraguay                        1675
alemania                        1571
italia                          1554
republica de corea               866
india                            781
japon                            699
portugal                         650
gran bretana                     623
canada                           616
el salvador                      577
federacion de rusia              534
f

### Reducir el numero de categorias encontrado en PAIS. 
- Arbitariamente, definamos solamente paises con mas de 1000 extranjeros 

In [38]:
def simplificaPais(text):
    text = text.split(' ')
    if 'peru' in text: text = 'peru'
    elif 'venezuela' in text: text= 'venezuela'
    elif 'haiti' in text: text= 'haiti'
    elif 'colombia' in text: text= 'colombia'
    elif 'bolivia' in text: text= 'bolivia'
    elif 'argentina' in text: text= 'argentina'
    elif 'ecuador' in text: text= 'ecuador'
    elif 'china' in text: text= 'china'
    elif "espana" in text: text= "espana"
    elif 'brasil' in text: text= 'brasil'
    elif 'dominicana' in text: text= 'dominicana'
    elif 'cuba' in text: text= 'cuba'
    elif 'estados' in text: text= 'eua'
    elif 'uruguay' in text: text= 'uruguay'
    elif 'mexico' in text: text= 'mexico'
    elif 'francia' in text: text= 'francia'
    elif 'paraguay' in text: text= 'paraguay'
    elif 'alemania' in text: text= 'alemania'
    elif 'italia' in text: text= 'italia'
    else: text = 'otros'
    return text 

In [39]:
X['PAIS'] = X.PAIS.apply(simplificaPais)

In [40]:
X.PAIS.value_counts()

peru          133949
venezuela     108870
haiti          77041
colombia       60876
bolivia        16164
argentina      15983
ecuador        15055
otros          13427
china           8738
espana          8200
brasil          7048
dominicana      5909
cuba            5467
eua             3931
uruguay         2905
mexico          2829
francia         2446
paraguay        1675
alemania        1571
italia          1554
Name: PAIS, dtype: int64

## 3. Preproceso para aplicación de modelos de ML

### Trabajando con Transformación de columnas, donde se mesclan tipos de datos (numericos y categóricos)

In [53]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

In [54]:
numeric_features = ['EDAD', 'ANIO']
categorical_features = ['PAIS', 'SEXO', 'ESTUDIOS', 'ACTIVIDAD', 'BENEFICIO']

### Creación de los pipelines de pre-processing para los datos categoricos y numericos

In [55]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)]) 

## 4. Testando vários modelos de ML

### 1. First Test using LogisticRegression

In [56]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs')

==> Model Score: 0.30 **Resultado: Malo**

In [ ]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'classifier__C': [0.1, 1.0, 10, 100],
}

grid_search = GridSearchCV(clf, param_grid, cv=10, iid=False)
grid_search.fit(X_train, y_train)

print(("best logistic regression from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

### 2. SVC

In [50]:
from sklearn.svm import LinearSVC
model = LinearSVC()

==> Model Score: 0.32 **Resultado: Malo** y muy demorado (mas de 1 hora)